In [99]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras import layers, models
from tensorflow.keras.layers import Layer, Input
from tensorflow.keras import backend as K
import pandas as pd

In [ ]:
def build_feature_extractor():
    base_model = model = ResNet50(weights=None, include_top=False, input_shape=(224, 224, 3))
    base_model.load_weights('resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5')
    base_model.trainable = False

    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(512, activation='relu')(x)
    x = Dense(256, activation='relu')(x)
    return models.Model(inputs=base_model.input, outputs=x)

feature_extractor = build_feature_extractor()

# Siamese Network

In [55]:
class EuclideanDistance(Layer):
    def __init__(self, **kwargs):
        super(EuclideanDistance, self).__init__(**kwargs)

    def call(self, inputs):
        x1, x2 = inputs
        return K.sqrt(K.sum(K.square(x1 - x2), axis=-1, keepdims=True))

In [56]:
def contrastive_loss(y_true, y_pred, margin=1.0):
    squared_pred = K.square(y_pred)  # D^2
    margin_square = K.square(K.maximum(margin - y_pred, 0))  # (margin - D)^2
    return K.mean((1 - y_true) * squared_pred + y_true * margin_square)

In [102]:
def triplet_loss(margin=0.2):
    def loss(y_true, y_pred):
        anchor, positive, negative = tf.split(y_pred, num_or_size_splits=3, axis=1)
        
        # Compute pairwise distances
        pos_dist = tf.reduce_sum(tf.square(anchor - positive), axis=1)
        neg_dist = tf.reduce_sum(tf.square(anchor - negative), axis=1)
        
        # Compute loss
        loss = tf.maximum(pos_dist - neg_dist + margin, 0.0)
        
        return tf.reduce_mean(loss)
    
    return loss

In [43]:
def SiameseNet():
    input_a = Input(shape=(224, 224, 3))
    input_b = Input(shape=(224, 224, 3))
    feature_a = feature_extractor(input_a)
    feature_b = feature_extractor(input_b)
    distance = EuclideanDistance()([feature_a, feature_b])
    siamese_model = models.Model(inputs=[input_a, input_b], outputs=distance)
    siamese_model.compile(loss=contrastive_loss, optimizer='adam', metrics=['accuracy'])
    return siamese_model


siamese_model = SiameseNet()
#siamese_model.summary()

In [101]:
dataset = pd.read_csv('lables.csv')
dataset.head()

,Label Number,Label Name
0,0,pink primrose
1,1,hard-leaved pocket orchid
2,2,canterbury bells
3,3,sweet pea
4,4,english marigold
